In [1]:
'''                                                 NAIVE BAYES
yeh bayes theorem pe based h
yeh maanti h ki saare columns independent h kisi column ko kisi bhi column se matalba nhi
used for text classification,sentiment analysis
yeh sirf classification k liye ues aati h 
isme hum spam classification bhi kr sakte h
bayes theorem: P(C|X)=P(X|C)*P(C)/P(X)

types of naive bayes:
-gaussian naive bayes(jab humare paas continous values ho)
-multinomial naive bayes(jab humare paas descrete features ho)
-Bernouli naive bayes(jab humare paas binary features ho)
'''

'                                                 NAIVE BAYES\nyeh bayes theorem pe based h\nyeh maanti h ki saare columns independent h kisi column ko kisi bhi column se matalba nhi\nused for text classification,sentiment analysis\nyeh sirf classification k liye ues aati h \nisme hum spam classification bhi kr sakte h\nbayes theorem: P(C|X)=P(X|C)*P(C)/P(X)\n\ntypes of naive bayes:\n-gaussian naive bayes(jab humare paas continous values ho)\n-multinomial naive bayes(jab humare paas descrete features ho)\n-Bernouli naive bayes(jab humare paas binary features ho)\n'

In [2]:
#spam and ham classification

In [3]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score,classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pickle,joblib
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
#yahha hum padeneg nlp in machine learning

In [4]:
x,y=[],[]
with open(r"C:\Users\Gargi\OneDrive\Desktop\data science\MACHINE LEARNING\Naive  Bayes\SMSSpamCollection.txt","r") as file:
    data=file.read()
    #iterate through each line
    for data_item in data.split("\n"):
        #it will check if the line contains exactly two item seperated by a tab
        if len(data_item.split("\t"))==2:
            #append the first element (lable )to y
            y.append(data_item.split("\t")[0])
            #append the second element (text) to x
            x.append(data_item.split("\t")[1])
    print("success")


success


In [5]:
#prep your data
data_dict={"mail_message":x,"label":y}
df=pd.DataFrame(data_dict)
df.head()

,mail_message,label
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10574 entries, 0 to 10573
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   mail_message  10574 non-null  object
 1   label         10574 non-null  object
dtypes: object(2)
memory usage: 165.3+ KB


In [7]:
for i in range(0,len(df)):
#this line uses a regular expression to remove anythong that isnt a letter
    message=re.sub("[^a-zA-Z]"," ",df["mail_message"][i])
    #txt data k sath thora sa preprocessing kr rahe h
    message=message.lower()
    df["mail_message"][i]=message
df.head()

C:\Users\Gargi\AppData\Local\Temp\ipykernel_34692\4011807164.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["mail_message"][i]=message


,mail_message,label
0,go until jurong point crazy available only ...,ham
1,ok lar joking wif u oni,ham
2,free entry in a wkly comp to win fa cup fina...,spam
3,u dun say so early hor u c already then say,ham
4,nah i don t think he goes to usf he lives aro...,ham


In [8]:
df["label"].value_counts()

label
spam    5747
ham     4827
Name: count, dtype: int64

In [9]:
ham=df[df["label"]=="ham"]
spam=df[df["label"]=="spam"].sample(4827)
df=pd.concat([ham,spam],axis=0)
df["label"].value_counts()

label
ham     4827
spam    4827
Name: count, dtype: int64

In [10]:
df["label"]=df["label"].map({"spam":1,"ham":0})
df.head()
df["label"].value_counts()

label
0    4827
1    4827
Name: count, dtype: int64

In [11]:
df.isnull().sum()

mail_message    0
label           0
dtype: int64

In [12]:
df.duplicated().sum()
#paye rhn do

3698

In [13]:
x=df["mail_message"]
y=df["label"]

In [14]:
#prepare the single list of mails
x=x.values
x[0:5]


array(['go until jurong point  crazy   available only in bugis n great world la e buffet    cine there got amore wat   ',
       'ok lar    joking wif u oni   ',
       'u dun say so early hor    u c already then say   ',
       'nah i don t think he goes to usf  he lives around here though',
       'even my brother is not like to speak with me  they treat me like aids patent '],
      dtype=object)

In [15]:
x

array(['go until jurong point  crazy   available only in bugis n great world la e buffet    cine there got amore wat   ',
       'ok lar    joking wif u oni   ',
       'u dun say so early hor    u c already then say   ', ...,
       'you   ve been selected for a free starbucks gift card  enter your number at www freedealz com to confirm ',
       'claim your free ringtone now by texting tone to        offer ends midnight  t c apply ',
       'free ringtone text first to       for a poly or text get to       for a true tone  help                  after  st free  tones are  x     pw to e  nd txt stop'],
      dtype=object)

In [16]:
#bag of words
#yeh text data ko numeric data me convert krta h
Bow_obj=CountVectorizer(
    max_features=2000,
    stop_words=stopwords.words("english")
)
x_transformed=Bow_obj.fit_transform(x).toarray()
print(x_transformed)
x_transformed.shape


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


(9654, 2000)

In [17]:
joblib.dump(Bow_obj,"vectorizer.lb")
#Ye ek function hai jo Python object ko save (serialize) karta hai kisi file me — jaise .pkl, .lb etc
#Tum CountVectorizer object (Bow_obj) ko "vectorizer.lb" file me save kar rahi ho
#taaki baad me bina dobara train kiye use load kar sako.

['vectorizer.lb']

In [18]:
x_train,x_test,y_train,y_test=train_test_split(
    x_transformed,y,test_size=0.2
)

In [19]:
BNB=BernoulliNB()
BNB.fit(x_train,y_train)

BernoulliNB()

In [20]:
BNB.score(x_test,y_test)
BNB.score(x_train,y_train)

0.9840735465492684

In [21]:
pred=BNB.predict(x_test)
pred


array([0, 0, 0, ..., 1, 0, 1], dtype=int64)

In [22]:
compare_dict={"actual":y_test,"pred":pred}
compare_df=pd.DataFrame(compare_dict)
compare_df.head()

,actual,pred
3912,0,0
1049,0,0
4032,0,0
1472,0,0
4274,0,0


In [23]:
pickle.dump(BNB,open("spamclassifiermodel.pkl","wb"))